<a href="https://colab.research.google.com/github/shashanksn111/AI-ML-Developer-Assignment-/blob/main/Task01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install cohere hnswlib unstructured

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [49]:
import cohere
import uuid
import hnswlib
from typing import List, Dict
from unstructured.partition.html import partition_html
from unstructured.chunking.title import chunk_by_title

co = cohere.Client("8mcORf0RaGpY10ha2goKWluRaLSOhs15NwgY6iOc")

In [50]:
raw_documents = [
    {
        "title": "Crafting Effective Prompts",
        "url": "https://docs.cohere.com/docs/crafting-effective-prompts"},
    {
        "title": "Advanced Prompt Engineering Techniques",
        "url": "https://docs.cohere.com/docs/advanced-prompt-engineering-techniques"},
    {
        "title": "Prompt Truncation",
        "url": "https://docs.cohere.com/docs/prompt-truncation"},
    {
        "title": "Preambles",
        "url": "https://docs.cohere.com/docs/preambles"}
]

In [55]:
!pip install nltk
import nltk

# Download the 'punkt' data package using the NLTK Downloader
# Specifying download_dir ensures it's saved to the desired location.
nltk.download('punkt', download_dir='/root/nltk_data/tokenizers/punkt/PY3_tab')

# Update the nltk data path
nltk.data.path.append('/root/nltk_data/tokenizers/punkt/PY3_tab')

class Vectorstore:
    def __init__(self, raw_documents: List[Dict[str, str]]):
        self.raw_documents = raw_documents
        self.docs = []
        self.docs_embs = []
        self.retrieve_top_k = 10
        self.rerank_top_k = 3
        # Call load_and_chunk, embed, and index within __init__
        self.load_and_chunk()
        self.embed()
        self.index()

    def load_and_chunk(self) -> None:
        """
        Loads the text from the sources and chunks the HTML content.
        """
        print("Loading documents...")

        for raw_document in self.raw_documents:
            elements = partition_html(url=raw_document["url"])
            chunks = chunk_by_title(elements)
            for chunk in chunks:
                self.docs.append(
                    {
                        "title": raw_document["title"],
                        "text": str(chunk),
                        "url": raw_document["url"],
                    }
                )

    def embed(self) -> None:
        """
        Embeds the document chunks using the Cohere API.
        """
        print("Embedding document chunks...")

        batch_size = 90
        self.docs_len = len(self.docs)
        for i in range(0, self.docs_len, batch_size):
            batch = self.docs[i : min(i + batch_size, self.docs_len)]
            texts = [item["text"] for item in batch]
            docs_embs_batch = co.embed(
                texts=texts, model="embed-english-v3.0", input_type="search_document"
            ).embeddings
            self.docs_embs.extend(docs_embs_batch)


    def index(self) -> None:
        """
        Indexes the documents for efficient retrieval.
        """
        print("Indexing documents...")

        self.idx = hnswlib.Index(space="ip", dim=1024)
        self.idx.init_index(max_elements=self.docs_len, ef_construction=512, M=64)
        self.idx.add_items(self.docs_embs, list(range(len(self.docs_embs))))

        print(f"Indexing complete with {self.idx.get_current_count()} documents.")

    def retrieve(self, query: str) -> List[Dict[str, str]]:
        """
        Retrieves document chunks based on the given query.

        Parameters:
        query (str): The query to retrieve document chunks for.

        Returns:
        List[Dict[str, str]]: A list of dictionaries representing the retrieved document chunks, with 'title', 'text', and 'url' keys.
        """

        # Dense retrieval
        query_emb = co.embed(
            texts=[query], model="embed-english-v3.0", input_type="search_query"
        ).embeddings

        doc_ids = self.idx.knn_query(query_emb, k=self.retrieve_top_k)[0][0]

        # Reranking
        rank_fields = ["title", "text"] # We'll use the title and text fields for reranking

        docs_to_rerank = [self.docs[doc_id] for doc_id in doc_ids]

        rerank_results = co.rerank(
            query=query,
            documents=docs_to_rerank,
            top_n=self.rerank_top_k,
            model="rerank-english-v3.0",
            rank_fields=rank_fields
        )

        doc_ids_reranked = [doc_ids[result.index] for result in rerank_results.results]



[nltk_data] Downloading package punkt to
[nltk_data]     /root/nltk_data/tokenizers/punkt/PY3_tab...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [56]:
vectorstore = Vectorstore(raw_documents)

Loading documents...
Embedding document chunks...
Indexing documents...
Indexing complete with 138 documents.


In [57]:
vectorstore.retrieve("Prompting by giving examples")

In [63]:
def run_chatbot(message, chat_history=[]):

    # Generate search queries, if any
    response = co.chat(message=message,
                        model="command-r-plus",
                        search_queries_only=True,
                        chat_history=chat_history)

    search_queries = []
    for query in response.search_queries:
        search_queries.append(query.text)

    # If there are search queries, retrieve the documents
    if search_queries:
        print("Retrieving information...", end="")

        # Retrieve document chunks for each query
        documents = []
        for query in search_queries:
            # Check if the result of retrieve is not None before extending
            retrieved_docs = vectorstore.retrieve(query)
            if retrieved_docs is not None:
                documents.extend(retrieved_docs)
            else:
                print(f"Warning: No documents found for query: {query}")  # Optional: Print a warning

        # Use document chunks to respond
        response = co.chat_stream(
            message=message,
            model="command-r-plus",
            documents=documents,
            chat_history=chat_history,
        )

    else:
        response = co.chat_stream(
            message=message,
            model="command-r-plus",
            chat_history=chat_history,
        )

    # Print the chatbot response and citations
    chatbot_response = ""
    print("\nChatbot:")

    for event in response:
        if event.event_type == "text-generation":
            print(event.text, end="")
            chatbot_response += event.text
        if event.event_type == "stream-end":
            if event.response.citations:
                print("\n\nCITATIONS:")
                for citation in event.response.citations:
                    print(citation)
            if event.response.documents:
                print("\nCITED DOCUMENTS:")
                for document in event.response.documents:
                    print(document)
            # Update the chat history for the next turn
            chat_history = event.response.chat_history

    return chat_history

In [64]:
start = 382
end = 397
text = 'similar vectors'
document_ids = ['doc_0', 'doc_2']

In [65]:
# Turn # 1
chat_history = run_chatbot("Hello, I have a question")


Chatbot:
Of course! I am here to help. Please go ahead with your question, and I will do my best to assist you.

In [66]:
# Turn # 2
chat_history = run_chatbot("What's the difference between zero-shot and few-shot prompting", chat_history)

Retrieving information...Warning: No documents found for query: zero-shot prompting

Chatbot:
Zero-shot and few-shot are both types of learning techniques used in machine learning and natural language processing. Here's the difference between the two:

**Zero-Shot Learning:**
Zero-shot learning refers to the ability of a model to perform a task without any explicit training examples from that task. In other words, the model has "zero shots" at learning from specific examples relevant to the task. Instead of relying on example input-output pairs, zero-shot learning relies on the model's ability to generalize and extrapolate from its pre-existing knowledge.

For example, if you have a language model trained on a large corpus of text, you might ask it to classify sentiment ("positive" or "negative") for a given sentence, even though it hasn't been trained specifically on sentiment classification examples. The model is expected to understand and perform the task based on its inherent knowl

In [67]:
# Turn # 3
chat_history = run_chatbot("How would the latter help?", chat_history)

Retrieving information...Warning: No documents found for query: benefits of few-shot learning

Chatbot:
Few-shot learning can be incredibly helpful in several ways:
1. **Data Efficiency:** One of the primary advantages of few-shot learning is its ability to learn from a small number of examples. In many real-world scenarios, labeled training data is expensive and time-consuming to acquire. Few-shot learning allows models to learn and generalize from just a handful of examples, making it more feasible to apply machine learning to a broader range of tasks.
2. **Adaptability:** Few-shot learning enables models to adapt quickly to new tasks. With just a few examples, the model can learn to perform a new classification or generation task. This adaptability is especially useful in dynamic environments where new categories or tasks emerge frequently.
3. **Reduced Training Time:** Training large machine learning models from scratch can take a significant amount of time and computational resour

In [68]:
# Turn # 4
chat_history = run_chatbot("What do you know about 5G networks?", chat_history)

Retrieving information...Warning: No documents found for query: 5G networks explained

Chatbot:
5G networks refer to the fifth generation of mobile communication technology, designed as the successor to 4G (LTE) networks. 5G brings significant improvements in terms of speed, latency, capacity, and connectivity compared to its predecessors. Here are some key aspects of 5G networks:

1. **Speed:** 5G offers much faster data transfer rates compared to 4G, with peak speeds reaching up to 10 gigabits per second (Gbps) or more. In practical terms, this means faster download and upload times, seamless streaming of high-resolution videos, and quicker file transfers.
2. **Low Latency:** Latency refers to the delay between sending and receiving data. 5G networks are designed to have extremely low latency, which can be as low as 1 millisecond or even less. This makes 5G ideal for applications that require real-time responsiveness, such as self-driving cars, industrial automation, and virtual real

In [69]:
print("Chat history:")
for c in chat_history:
    print(c, "\n")

Chat history:
role='USER' message='Hello, I have a question' tool_calls=None 

role='CHATBOT' message='Of course! I am here to help. Please go ahead with your question, and I will do my best to assist you.' tool_calls=None 

role='USER' message="What's the difference between zero-shot and few-shot prompting" tool_calls=None 

role='CHATBOT' message='Zero-shot and few-shot are both types of learning techniques used in machine learning and natural language processing. Here\'s the difference between the two:\n\n**Zero-Shot Learning:**\nZero-shot learning refers to the ability of a model to perform a task without any explicit training examples from that task. In other words, the model has "zero shots" at learning from specific examples relevant to the task. Instead of relying on example input-output pairs, zero-shot learning relies on the model\'s ability to generalize and extrapolate from its pre-existing knowledge.\n\nFor example, if you have a language model trained on a large corpus of